# 0. Install and Import Dependencies

In [14]:
# !pip install mediapipe

In [15]:
# !pip install opencv-python

In [17]:
# importing OpenCV
import cv2

# importing mediapip for pose estimation libraries and different mediapipe solutions.
import mediapipe as mp

# importing numpy
import numpy as np

# mp_drawing will help in visualizing poses.
mp_drawing = mp.solutions.drawing_utils

# mp_pose is importing our pose estimation model.
mp_pose = mp.solutions.pose

In [13]:
# # Capturing Video using cap and 0 no. is for my webcam.
# cap = cv2.VideoCapture(0)

# # Looping through particular feed
# while cap.isOpened():
#     # ret is return variable frame is giving the image of webcam and storing it in ret.
#     ret, frame = cap.read()

#     #gives the pop-up on our screen which will help us in visualize.(boxname, frame(cathcing image from webcam))
#     cv2.imshow('Mediapipe Feed', frame)
    
#     # breaking out of our feed.
    
#     if cv2.waitKey(10) & 0xFF == ord('q'):
#         break

# # Release video capture device. 
# cap.release()

# # This will close our feed.
# cv2.destroyAllWindows()

-->ord('q') returns the Unicode code point of q

-->cv2.waitkey(1) returns a 32-bit integer corresponding to the pressed key

-->& 0xFF is a bit mask which sets the left 24 bits to zero, because ord() returns a value betwen 0 and 255, since your keyboard only has a limited character set

-->Therefore, once the mask is applied, it is then possible to check if it is the corresponding key.

Unicode Character “q” (U+0071)

# 1. Make Detections

In [28]:
cap = cv2.VideoCapture(0)
# Setingup mediapipe instance (mdc & mtc = we can increase both for very high accuracy for our model but sometimes setting up too can decrease detection of model so for now I'm keeping it at default.) as veriable pose.

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Applyinig performance training (This will save bunch of memories once we this to our pose estimation model).
        image.flags.writeable = False
      
        # Make detection and storing it into results.
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detections(drawing our image using our libraries, Pose Landmark Model The landmark model in MediaPipe Pose predicts the location of 33 pose landmarks. POSE_CONNECTIONS = which landmark is connected connected to which)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [24]:
# results.pose_landmarks

In [22]:
# mp_pose.POSE_CONNECTIONS

In [29]:
# mp_drawing.DrawingSpec??

# 2. Determining Joints

<img src="https://google.github.io/mediapipe/images/mobile/pose_tracking_full_body_landmarks.png" style="height:300px" >

In [31]:
# cap = cv2.VideoCapture(0)
# ## Setup mediapipe instance
# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#     while cap.isOpened():
#         ret, frame = cap.read()
        
#         # Recolor image to RGB
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False
      
#         # Make detection
#         results = pose.process(image)
    
#         # Recolor back to BGR
#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
#         # Extract landmarks(If we don't make any detection then we can just pass)
#         try:
#             landmarks = results.pose_landmarks.landmark
#             # print(landmarks)
#         except:
#             pass
        
        
#         # Render detections
#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
#                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
#                                  )               
        
#         cv2.imshow('Mediapipe Feed', image)

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

There are 33 landmarks in total, starting from index 0. These represent the different joints within the pose.

In [33]:
# len(landmarks)

In [35]:
# for lndmrk in mp_pose.PoseLandmark:
#     print(lndmrk)

In [45]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.7621892690658569
y: 0.9104772210121155
z: -0.24122849106788635
visibility: 0.9988343715667725

In [48]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.7986907958984375
y: 1.2198669910430908
z: -0.1517091542482376
visibility: 0.22890694439411163

In [47]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 0.7882274389266968
y: 1.486548900604248
z: -0.4084400534629822
visibility: 0.2883382737636566

# 3. Calculate Angles

In [50]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    # Calculating radians for particular joints.
    # The numpy. arctan2() method computes element-wise arc tangent of arr1/arr2 choosing the quadrant correctly.
       
        # subtracting y values 
                # {
                # (y from our endpoint - y from out midpoint c[1]-b[1],
                    #  [ landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value] (y: 1.486548900604248), landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value] (y: 1.2198669910430908) ]),
                    
                # (x from our endpoint - x from out midpoint c[0]-b[0],
                    # [ landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value] (x: 0.7882274389266968), landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value] (x: 0.7986907958984375) ]),

                # (y from our firstpoint - y from out midpoint a[1]-b[1],
                    # [ landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value] (y: 0.9104772210121155), landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value] (y: 1.2198669910430908) ]),

                # (x from our firstpoint - x from out midpoint a[0]-b[0],
                    # [ landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value] (x: 0.7621892690658569), landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value] (x: 0.7986907958984375)])
                # }
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])

    # Converting into 360 degree angle.
    angle = np.abs(radians*180.0/np.pi)
    
    # Calculating angle between 0 - 180.
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [8]:
# Creating variable fo shoulder, elbow and wrist and passing x,y in it.

shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]

wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [28]:
shoulder, elbow, wrist

([0.7351917028427124, 0.9729941487312317],
 [0.859039306640625, 1.367794156074524],
 [0.8037980198860168, 1.6398569345474243])

In [29]:
calculate_angle(shoulder, elbow, wrist)

151.10586199108832

In [30]:
tuple(np.multiply(elbow, [640, 480]).astype(int))

(549, 656)

In [9]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# 4. Curl Counter

In [10]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1
2
3
4
5
